In [1]:
import braindecode

In [2]:
import mne
from scipy.io import loadmat
import scipy
import sklearn
import numpy as np
import pandas as pd
import glob
from mne.decoding import CSP
import os

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda

In [4]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [5]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

In [6]:
n_jobs = None  # for multicore parallel processing, set it to 1 if cause memory issues, for full utilization set to -1

## Data Loading and Conversion to MNE Datatypes


In [7]:
current_folder = globals()['_dh'][0]  # a hack to get path of current folder in which jupyter file is located
data_path = os.path.join(current_folder, r"C:\Users\User\Documents\GitHub\Frequency-Adaptive-Temporal-Kernel-EEGNet\Data")

In [8]:
training_files   = glob.glob(data_path + '/*T.mat')
len(training_files)     # if  return zero,then no file is loaded

10

In [9]:
def get_mne_epochs(filepath, verbose=verbose, t_start=2, fs=512, mode='train'):
    '''
    This function reads the EEG data from .mat file and convert it to MNE-Python Compatible epochs
    data structure. It takes data from [0, 8] sec range and return it by setting t = 0 at cue onset
    i.e. 3 seconds and dropping first two seconds so the output data is in [-1.0, 5.0] sec range. The
    Details can be found in the preprocessing section of the attached document
    '''
    mat_data = loadmat(filepath) # read .mat file
    eeg_data= mat_data['RawEEGData']
    idx_start = fs*t_start      
    eeg_data = eeg_data[:, :, idx_start:]
    event_id = {'left-hand': 0, 'right-hand': 1} # pytorch expects labels in [0, n_classes-1]
    channel_names = ['F3', 'FC3', 'C3', 'CP3', 'P3', 'FCz', 'CPz', 'F4', 'FC4', 'C4', 'CP4', 'P4']
    info = mne.create_info(ch_names=channel_names, sfreq=fs, ch_types='eeg')
    epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=t_start-3.0)
    epochs.set_montage('standard_1020')
    epochs.filter(1., None) 
    epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction
    
    if mode == 'train': # this in only applicable for training data
        epochs.event_id = event_id 
        epochs.events[:,2] = mat_data['Labels'].ravel() - 1    
    return epochs 

def get_labels(filepath):
    mat_data = loadmat(filepath) # read .mat file
    return mat_data['Labels'].ravel() - 1

In [10]:
epochs, labels = get_mne_epochs(training_files[0], verbose=verbose), get_labels(training_files[0])
data = epochs.get_data()
print('Shape of EEG Data: ', data.shape, '\t Shape of Labels: ', labels.shape) 

Shape of EEG Data:  (80, 12, 3072) 	 Shape of Labels:  (80,)


### Training Data

In [11]:
# loading original data
epochs_list_train = []
for i in training_files:
    epochs_list_train.append(get_mne_epochs(i, verbose=verbose))

## Deep Learning with Braindecode 

### It's Training Time with [0.5, 4.5] sec and 2sec window with 125ms stride

In [12]:
from braindecode.datautil import create_from_mne_epochs

window_size = 1024   # 2 sec windows
window_stride = 64   # 125 ms stride

windows_datasets_list = []

for epoch in epochs_list_train:
    # Create windows per subject
    windows_dataset = create_from_mne_epochs(
        [epoch.crop(tmin=0.5, tmax=4.5, include_tmax=False)],
        window_size_samples=window_size,
        window_stride_samples=window_stride,
        drop_last_window=False
    )
    # Add labels as a separate attribute
    windows_dataset.update_description = pd.DataFrame(
        data=np.concatenate([d.y for d in windows_dataset.datasets]),
        columns=['labels']
    )
    windows_datasets_list.append(windows_dataset)

print("Datasets:", len(windows_datasets_list))
print("Total windows per subject:", len(windows_datasets_list[0]))


Datasets: 10
Total windows per subject: 1360


In [13]:
from braindecode.preprocessing import exponential_moving_standardize

low_cut_hz = 8.   # low cut frequency for filtering
high_cut_hz = 32. # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

def custom_exp_moving_std_fn(epochs, factor_new=factor_new, init_block_size=init_block_size):
    """Apply exponential moving standardization to MNE epochs inplace."""
    data = epochs.get_data()
    for i in range(len(data)):
        data[i] = exponential_moving_standardize(
            data[i], factor_new=factor_new, init_block_size=init_block_size
        )
    epochs._data = data
    return epochs

# Apply preprocessing to each dataset
for windows_dataset in windows_datasets_list:
    # Extract the underlying MNE Epochs object
    epochs = windows_dataset.datasets[0].windows
    epochs.load_data()  # Ensure data is loaded into memory

    # 1) Keep only EEG channels
    epochs.pick_types(eeg=True)

    # 2) Bandpass filter
    epochs.filter(l_freq=low_cut_hz, h_freq=high_cut_hz)

    # 3) Exponential moving standardization
    custom_exp_moving_std_fn(epochs, factor_new=factor_new, init_block_size=init_block_size)


In [14]:
batch_size = 32 #64
n_epochs = 25 #25 #20 #25 use few epochs for quick verification

In [15]:
# Creating a model
import torch
from braindecode.util import set_random_seeds
from Frequency_Adaptive_model import AdaptiveEEGNet  

cuda = torch.cuda.is_available()  
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True

seed = 20200220  # random seed to make results reproducible
set_random_seeds(seed=seed, cuda=cuda)

# Deterministic training setup 
torch.backends.cudnn.deterministic = True 
torch.backends.cudnn.benchmark = False


# Model hyperparameters

n_classes = 2
n_chans = windows_datasets_list[0][0][0].shape[0]      # EEG channels
input_window_samples = windows_datasets_list[0][0][0].shape[1]  # time samples

# Instantiate AdaptiveEEGNet
model = AdaptiveEEGNet(
    nb_classes=n_classes,
    Chans=n_chans,
    Samples=input_window_samples,
    kernLength=128,
    F1=16,
    D=2,
    F2=32,
    dropoutRate=0.3,
    sample_rate=512
)

# Send model to GPU
if cuda:
    model.cuda()


In [16]:
import torch
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
import numpy as np

def training_within_subject_adaptive(windows_datasets_list, model_class, n_epochs=25, batch_size=32, val_ratio=0.2):
    """
    Trains AdaptiveEEGNet model separately for each subject (within-subject classification).
    Reports best validation accuracy per subject and mean accuracy across all subjects.
    """
    all_subject_acc = []

    for subj_idx, subj_data in enumerate(windows_datasets_list):
        print(f"\n=== Subject {subj_idx+1}/{len(windows_datasets_list)} ===")

        # Gather data
        X = np.concatenate([ds.windows for ds in subj_data.datasets], axis=0)
        y = np.concatenate([ds.y for ds in subj_data.datasets], axis=0)

        # Split train/validation
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=val_ratio, stratify=y, random_state=42
        )

        # Convert to tensors
        train_tensor = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
        val_tensor   = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))

        # Create a fresh model for each subject
        n_chans = X.shape[1]
        input_window_samples = X.shape[2]
        n_classes = len(np.unique(y))

        model = model_class(
            nb_classes=n_classes,
            Chans=n_chans,
            Samples=input_window_samples,
            kernLength=128,
            F1=16,
            D=2,
            F2=32,
            dropoutRate=0.3,
            sample_rate=512
        )

        if torch.cuda.is_available():
            model.cuda()
            device = 'cuda'
        else:
            device = 'cpu'

        # Create classifier
        clf = EEGClassifier(
            model,
            criterion=torch.nn.CrossEntropyLoss,
            optimizer=torch.optim.AdamW,
            optimizer__lr=0.02,
            optimizer__weight_decay=0.0005,
            batch_size=batch_size,
            train_split=predefined_split(val_tensor),  # validation data
            callbacks=[("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs-1))],
            device=device
        )

        # Train model
        clf.fit(train_tensor, y=y_train, epochs=n_epochs)

        # Retrieve best validation accuracy
        for name, cb in clf.callbacks_:
            if name == "valid_acc":
                best_val_acc = cb.best_score_
                break

        print(f"Best Validation Accuracy for Subject {subj_idx+1}: {best_val_acc:.3f}")
        all_subject_acc.append(best_val_acc)

    mean_acc = np.mean(all_subject_acc)
    print("\n=== Mean Within-Subject Accuracy: {:.3f} ===".format(mean_acc))
    return all_subject_acc, mean_acc


In [17]:
training_within_subject_adaptive(windows_datasets_list, AdaptiveEEGNet, n_epochs=25, batch_size=32)



=== Subject 1/10 ===
  epoch    train_loss    valid_acc    valid_loss      lr     dur
-------  ------------  -----------  ------------  ------  ------
      1        0.8624       0.4853        1.2243  0.0200  2.1930
      2        0.6579       0.6066        0.8830  0.0199  2.2491
      3        0.6659       0.7022        0.5522  0.0197  2.2423
      4        0.5964       0.5147        4.0993  0.0192  2.2323
      5        0.5749       0.6544        0.6064  0.0187  2.2847
      6        0.5692       0.5662        0.7795  0.0179  2.1823
      7        0.5845       0.6213        1.0806  0.0171  2.1320
      8        0.4741       0.5147        2.8854  0.0161  2.1585
      9        0.4852       0.6103        1.8598  0.0150  2.1768
     10        0.4419       0.5147        5.2380  0.0138  2.1378
     11        0.4088       0.5000        9.4527  0.0126  2.1232
     12        0.5036       0.5000       13.9835  0.0113  2.2840
     13        0.4527       0.5147        3.2530  0.0100  2.2087
   

([0.9375,
  1.0,
  0.9411764705882353,
  0.8235294117647058,
  0.9522058823529411,
  0.9669117647058824,
  0.9816176470588235,
  0.9375,
  0.9632352941176471,
  0.9338235294117647],
 np.float64(0.94375))